# YOLOv3 To Detect and Crop Image From KeyFrame

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob 

from google.colab import drive, files
drive.mount('/content/drive/')
path = "/content/drive/MyDrive/ANN-Project/"

Mounted at /content/drive/


In [ ]:
classes = None
with open(path+'coco.names', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# read pre-trained model and config file
net = cv2.dnn.readNet(path+'yolov3.weights', path+'yolov3.cfg')

Output Txt File

In [ ]:
file1 = open(path+"4-5.txt","a+")

data_path = os.path.join(path+'4-5/','*g') 
files = glob.glob(data_path) 

In [ ]:
for f1 in files:
  image = plt.imread(f1)
  
  Width = image.shape[1]
  Height = image.shape[0]
  
  # create input blob 
  # set input blob for the network
  net.setInput(cv2.dnn.blobFromImage(image, 0.00392, (416,416), (0,0,0), True, crop=False))

  # run inference through the network
  # and gather predictions from output layers

  layer_names = net.getLayerNames()
  output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
  outs = net.forward(output_layers)

  class_ids = []
  confidences = []
  boxes = []

  #create bounding box 
  for out in outs:
    for detection in out:
      scores = detection[5:]
      class_id = np.argmax(scores)
      confidence = scores[class_id]
      if confidence > 0.5:
        center_x = int(detection[0] * Width)
        center_y = int(detection[1] * Height)
        w = int(detection[2] * Width)
        h = int(detection[3] * Height)
        x = center_x - w / 2
        y = center_y - h / 2
        class_ids.append(class_id)
        confidences.append(float(confidence))
        boxes.append([x, y, w, h])
  
  indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.1, 0.1)

  #check if is people detection
  for i in indices:
    i = i[0]
    box = boxes[i]
    if class_ids[i]==0:
      listToStr = ' '.join(map(str, box))
      file1.writelines(os.path.basename(f1)+' '+listToStr+'\n')

file1.close()

Read Bounding Box Data from Txt File to Crop Image

In [ ]:
file1 = open(path+"4-5.txt","r")
data = file1.readlines()
print(data)
file1.close()

['0147 263.5 159.0 221 260\n', '0206 131.5 159.5 153 257\n', '0206 362.0 126.5 202 301\n', '1146 436.5 136.0 193 292\n', '1146 80.0 166.5 142 247\n', '1111 129.5 141.5 193 275\n', '1081 117.5 152.5 165 267\n', '0061 145.5 136.5 137 283\n', '0061 278.5 184.5 261 249\n', '0173 353.5 155.0 213 264\n', '0173 129.5 239.0 163 162\n', '0203 123.5 152.0 155 262\n', '0203 363.5 129.0 181 290\n', '0318 357.0 155.5 194 269\n', '0321 291.0 160.0 276 254\n', '0794 120.0 21.0 186 246\n', '0794 292.5 141.5 171 277\n', '0723 235.0 244.0 164 142\n', '1078 107.5 154.5 165 267\n', '1078 359.5 122.0 187 296\n', '0046 78.0 147.5 150 269\n', '0046 419.5 133.0 193 286\n', '0806 115.5 33.5 191 213\n', '0261 369.5 17.5 61 177\n', '0044 94.0 96.5 92 341\n', '0044 421.5 139.5 191 277\n', '0402 106.0 163.5 144 255\n', '0402 288.5 193.0 271 222\n', '1101 115.0 145.0 142 272\n', '1101 389.5 137.0 169 282\n', '0192 154.5 158.0 135 262\n', '0998 174.5 158.0 133 264\n', '0998 284.0 126.0 196 294\n', '0174 152.0 220.5 

In [ ]:
import random

Crop Avatars

In [ ]:
for x in data:
    x = x.split()
    print(x)
    new_img = plt.imread(path+"4-5/frame"+x[0]+".jpg")

    y_min = abs(int( float(x[2]) ))
    y_max = abs(int( float(x[2])+float(x[4]) ))
    x_min = abs(int( float(x[1]) ))
    x_max = abs(int( float(x[1])+float(x[3]) ))

    crop_img = new_img[y_min:y_max, x_min:x_max]
    plt.imsave(path+"4-5-Crop/"+x[0]+"-"+"{:02d}.jpg".format(random.randint(1, 9)), crop_img)

['0147', '263.5', '159.0', '221', '260']
['0206', '131.5', '159.5', '153', '257']
['0206', '362.0', '126.5', '202', '301']
['1146', '436.5', '136.0', '193', '292']
['1146', '80.0', '166.5', '142', '247']
['1111', '129.5', '141.5', '193', '275']
['1081', '117.5', '152.5', '165', '267']
['0061', '145.5', '136.5', '137', '283']
['0061', '278.5', '184.5', '261', '249']
['0173', '353.5', '155.0', '213', '264']
['0173', '129.5', '239.0', '163', '162']
['0203', '123.5', '152.0', '155', '262']
['0203', '363.5', '129.0', '181', '290']
['0318', '357.0', '155.5', '194', '269']
['0321', '291.0', '160.0', '276', '254']
['0794', '120.0', '21.0', '186', '246']
['0794', '292.5', '141.5', '171', '277']
['0723', '235.0', '244.0', '164', '142']
['1078', '107.5', '154.5', '165', '267']
['1078', '359.5', '122.0', '187', '296']
['0046', '78.0', '147.5', '150', '269']
['0046', '419.5', '133.0', '193', '286']
['0806', '115.5', '33.5', '191', '213']
['0261', '369.5', '17.5', '61', '177']
['0044', '94.0', '96.5

# OpenPose To Estimate Pose of Avatars

In [ ]:
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`

Selecting previously unselected package libgflags2.2.
(Reading database ... 160772 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.3.5-1) ...
Selecting previously unselected package libhdf5-serial-dev.
Preparing to unpack .../04-libhdf5-serial-dev_1.10.0-patch1+docs-4_all.deb ...
Unpacking libhdf5-serial-dev (1.10.0-patch1+docs-4) ...
Selecting previously unselected package libleveldb1v5:amd64.
Preparing to unpack ...

Read Image Directory and Estimate Pose to Output json in local temporary directory

In [ ]:
!cd openpose && ./build/examples/openpose/openpose.bin --display 0 --image_dir ../drive/MyDrive/ANN-Project/punches/ --write_json ./output/ --render_pose 0

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 1093.146892 seconds.


In [ ]:
import json

Output Txt File For Pose Data

In [ ]:
output_file = open(path+"dataset.txt","a+")

In [ ]:
classId = '1'

for pose_file in os.listdir('./openpose/output/'):
  # Opening JSON file
  f = open("./openpose/output/"+pose_file,)
    
  # returns JSON object as 
  # a dictionary
  data = json.load(f)
    
  # Iterating through the json
  # list
  if len(data['people']) != 0:
    temp = data['people'][0]
    listToStr = ','.join(map(str, temp['pose_keypoints_2d']))
    output_file.writelines(os.path.basename(pose_file)[0:5]+".jpg"+','+classId+','+listToStr+'\n')

  # Closing file
  f.close()

output_file.close()